In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Read the CSV file into a pandas DataFrame
correct_form = pd.read_csv("/content/drive/MyDrive/Powerlift Phone/Training Data - Correct Curl.csv")

# Extract values from the first column
first_column_values = correct_form.iloc[:, 0]

# Extract text after the last space for each value
processed_values = first_column_values.apply(lambda x: x.split()[-1] if isinstance(x, str) else x)

# Update the DataFrame with the processed values
correct_form.iloc[:, 0] = processed_values

# NOW WITH INCORRECT VALUES
# Read the CSV file into a pandas DataFrame
incorrect_form = pd.read_csv("/content/drive/MyDrive/Powerlift Phone/Training Data - Incorrect Movement - CSV.csv")

# Extract values from the first column
incorrect_first_column_values = incorrect_form.iloc[:, 0]

# Extract text after the last space for each value
processed_values = incorrect_first_column_values.apply(lambda x: x.split()[-1] if isinstance(x, str) else x)

# In processed_values, delete the first character of each element
processed_values = processed_values.str[1:]

# Update the DataFrame with the processed values
incorrect_form.iloc[:, 0] = processed_values

# In incorrect_form, delete the last character of every element in the last column
incorrect_form.iloc[:, -1] = incorrect_form.iloc[:, -1].str[:-1]

In [ ]:
print("CORRECT FORM: ")
print(correct_form)
print("------------------------------")
print("INCORRECT FORM:")
print(incorrect_form)

In [3]:
import numpy as np
import tensorflow as tf

X = np.asarray(pd.concat([correct_form, incorrect_form], axis=0, ignore_index=True).astype(float))

Y = np.concatenate([np.ones(len(correct_form)), np.zeros(len(incorrect_form))])

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, shuffle= True)

In [5]:
from tensorflow.keras import models, layers

def build_binary_classification_model(input_size):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_dim=input_size))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification requires a sigmoid activation
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model 1
model_1 = build_binary_classification_model(input_size=34)
model_1.fit(x_train, y_train, epochs=10, batch_size=32)

# Model 2 with a different architecture
model_2 = models.Sequential()
model_2.add(layers.Dense(128, activation='relu', input_dim=34))
model_2.add(layers.Dense(64, activation='relu'))
model_2.add(layers.Dense(1, activation='sigmoid'))
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
98/98 [==============================] - 2s 4ms/step - loss: 12.9337 - accuracy: 0.8509
Epoch 2/10
98/98 [==============================] - 0s 3ms/step - loss: 2.4048 - accuracy: 0.9367
Epoch 3/10
98/98 [==============================] - 0s 4ms/step - loss: 1.6454 - accuracy: 0.9444
Epoch 4/10
98/98 [==============================] - 0s 4ms/step - loss: 1.3555 - accuracy: 0.9499
Epoch 5/10
98/98 [==============================] - 0s 4ms/step - loss: 0.9898 - accuracy: 0.9573
Epoch 6/10
98/98 [==============================] - 0s 3ms/step - loss: 0.7622 - accuracy: 0.9579
Epoch 7/10
98/98 [==============================] - 0s 4ms/step - loss: 0.8073 - accuracy: 0.9605
Epoch 8/10
98/98 [==============================] - 0s 3ms/step - loss: 0.6588 - accuracy: 0.9621
Epoch 9/10
98/98 [==============================] - 0s 4ms/step - loss: 0.8757 - accuracy: 0.9582
Epoch 10/10
98/98 [==============================] - 0s 2ms/step - loss: 0.5585 - accuracy: 0.9676
Epoch 1/10
98/98 [

In [6]:
loss_1, accuracy_1 = model_1.evaluate(x_test, y_test)
print(f'Model 1 - Test Accuracy: {accuracy_1 * 100:.2f}%')

loss_2, accuracy_2 = model_2.evaluate(x_test, y_test)
print(f'Model 2 - Test Accuracy: {accuracy_2 * 100:.2f}%')


18/18 [==============================] - 0s 2ms/step - loss: 1.3159 - accuracy: 0.9509
Model 1 - Test Accuracy: 95.09%
18/18 [==============================] - 0s 2ms/step - loss: 1.8488 - accuracy: 0.9364
Model 2 - Test Accuracy: 93.64%


In [10]:
converter_1 = tf.lite.TFLiteConverter.from_keras_model(model_1)
tflite_model_1 = converter_1.convert()
with open('model_1.tflite', 'wb') as f:
    f.write(tflite_model_1)

# Convert and save model 2
converter_2 = tf.lite.TFLiteConverter.from_keras_model(model_2)
tflite_model_2 = converter_2.convert()
with open('model_2.tflite', 'wb') as f:
    f.write(tflite_model_2)

In [18]:


print(x_train[0])

[   0.         0.       360.4414   137.40208    0.         0.
  480.23563  152.89265    0.         0.       604.53687  470.27496
  335.33374  488.8492   653.7289   837.8279     0.         0.
  589.8475  1138.1897     0.         0.       554.59985 1127.2255
  400.2505  1115.6862   582.7978  1453.4626     0.         0.
    0.         0.       466.42474 1585.9644 ]
